In [14]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings

## Load the documents

In [11]:
import pathlib

p = pathlib.Path('/Users/manuel.mosquera/Documents/projects/mercado_libre_project')
list(p.glob('**/*.tsx'))

[PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/searchBar/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/searchBar/searchBar.test.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productsList/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productsList/productsList.test.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productDetail/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productDetail/productDetail.test.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/breadcrumb/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/breadcrumb/breadcrumb.test.tsx'),
 PosixPath('/Users/manuel.mo

In [28]:
directory_path = '/Users/manuel.mosquera/Documents/projects/mercado_libre_project'
loader = DirectoryLoader(directory_path, loader_cls=TextLoader, glob='**/*.tsx')
docs = loader.load()

In [29]:
len(docs)

16

In [30]:
print(docs[0].page_content)

import { useState, FC, SyntheticEvent } from "react";
import Image from "next/image";
import SearchIcon from "../../common/icons/search";

import styles from "./searchBar.styles.module.scss";

interface Props {
  placeholder?: string;
  onClick(val: string): void;
}

const SearchBar: FC<Props> = ({ placeholder, onClick }) => {
  const [search, setSearch] = useState("");

  const handleSubmit = (event: SyntheticEvent) => {
    event.preventDefault();
    onClick(search);
  };

  return (
    <header className={styles.container}>
      <Image
        data-testid="logo"
        src="/images/mercado-libre-logo.png"
        alt="Mercado libre logo"
        width="90"
        height="90"
      />
      <form
        className={styles.searchForm}
        onSubmit={handleSubmit}
        data-testid="form"
      >
        <input
          className={styles.searchInput}
          placeholder={placeholder}
          value={search}
          onChange={(e) => setSearch(e.target.value)}
          da

In [40]:
import re

preprocess_docs = [{'file': doc.page_content, 'file_path': re.match(directory_path + r'\/(.*)', doc.metadata['source']).group(1) } for doc in docs if doc.page_content is not None]

In [41]:
for p in preprocess_docs:
  print(p['file_path'])

src/components/searchBar/index.tsx
src/components/searchBar/searchBar.test.tsx
src/components/productsList/index.tsx
src/components/productsList/productsList.test.tsx
src/components/productDetail/index.tsx
src/components/productDetail/productDetail.test.tsx
src/components/breadcrumb/index.tsx
src/components/breadcrumb/breadcrumb.test.tsx
src/common/icons/search.tsx
src/pages/index.tsx
src/pages/items.tsx
src/pages/_document.tsx
src/pages/_app.tsx
src/pages/api/items.tsx
src/pages/api/items/[id].tsx
src/pages/items/[id].tsx


## Generate a document description

In [77]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is the purpose of the following file in a project?\nGive me a detail description of the file \nfile path: {file_path} \nfile content: {file}"

llm = OpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
files_description = llm_chain.apply(preprocess_docs)

In [78]:
len(files_description)

16

In [79]:
print(files_description[0]['text'])


The file src/components/searchBar/index.tsx is a React component that provides the user with a search bar. It has a logo, input field, and search button. It allows the user to enter a search query in the input field and then submit the query (by pressing the search button or by hitting enter on the keyboard). When the user submits the query, the onClick function is called, which can be used to trigger an API call to fetch data based on the query. The component also has a default placeholder text that can be overridden by passing in a placeholder prop.


## Create an embedding with the description

In [80]:
desciptions_to_embed =  [f'File path: {preprocess_docs[i]["file_path"]}\nFile description: {files_description[i]["text"]}' for i in range(len(files_description))]

In [81]:
print(desciptions_to_embed[0])

File path: src/components/searchBar/index.tsx
File description: 
The file src/components/searchBar/index.tsx is a React component that provides the user with a search bar. It has a logo, input field, and search button. It allows the user to enter a search query in the input field and then submit the query (by pressing the search button or by hitting enter on the keyboard). When the user submits the query, the onClick function is called, which can be used to trigger an API call to fetch data based on the query. The component also has a default placeholder text that can be overridden by passing in a placeholder prop.


In [82]:
embeddings_model = OpenAIEmbeddings(disallowed_special=set([]))

## Create the vector store

In [83]:
from langchain.vectorstores import FAISS


db = FAISS.from_texts(desciptions_to_embed, embeddings_model)

## Create the steps to solve a task

In [84]:
task = "Add a pagination feature to the items list, listing 2 items per page."

In [85]:
retriever = db.as_retriever()

In [86]:
relevant_docs = retriever.get_relevant_documents(task)

In [87]:
most_relevant_docs = relevant_docs[:5]

In [88]:
most_relevant_docs = '\n\n'.join([file.page_content for file in most_relevant_docs])

In [89]:
print(most_relevant_docs)

File path: src/pages/items.tsx
File description: 
This file is a React component that is used to display a list of items. The component uses the Next.js framework to render content on the server-side, as well as to access the router and head components for SEO purposes. It imports the items.tsx file from the api folder, which contains the response from the server with the items that match the search query. It also imports components such as Breadcrumb, ProductsList, and Head to render the list of items along with the search query, categories filter, and SEO information. The component also uses the useCallback hook to set up a callback function to handle the click event on the breadcrumbs, which will update the URL query parameters with the selected category.

File path: src/pages/api/items.tsx
File description: 
The src/pages/api/items.tsx file is an API endpoint for a project. The endpoint is responsible for fetching data from the Mercado Libre API and returning the response as a JSON

In [92]:
import openai
import os

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

In [93]:
chat_plan = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"Give me a detailed description of how I can accomplish the following task: {task}. This is some information of the files where I will be working: {most_relevant_docs}"}])

In [94]:
print(chat_plan.choices[0].message.content)

To add a pagination feature to the items list in the "src/pages/items.tsx" file, follow these steps:

1. Import the necessary components and libraries:
   - Import the Pagination component from the library or create a custom Pagination component.
   - Import the useState and useEffect hooks from React.

2. Set up the necessary state variables:
   - Declare a state variable "currentPage" to store the current page number. Initialize it with 1.
   - Declare a state variable "itemsPerPage" to store the number of items per page. Initialize it with 2.
   - Declare a state variable "totalItems" to store the total number of items in the list. Initialize it with the length of the items list.

3. Calculate the necessary values:
   - Calculate the total number of pages by dividing the total number of items by the number of items per page. Use the Math.ceil() function to round up to the nearest whole number.
   - Calculate the index of the first item on the current page by subtracting 1 from the c